# Apprentissage de représentation, Texte et recommendation

## Partie B : Apprentissage de représentation, Fine-Tuning et Convolutions

Nicolas Baskiotis (nicolas.baskiotis@sorbonne-univeriste.fr) Benjamin Piwowarski (benjamin.piwowarski@sorbonne-universite.fr) -- MLIA/ISIR, Sorbonne Université

In [12]:
import torchtext
import torchdata # restart kernel after intall
import torch
assert torchtext.__version__ >= "0.11.0"
from torchtext.datasets import IMDB # recuperation directe des données
from torch.nn.utils.rnn import pad_sequence
from torchtext.data.utils import get_tokenizer
from torch.utils.data import  DataLoader
from torch import nn
from torch.utils.tensorboard import SummaryWriter
import time
from tqdm import tqdm
import os
TB_PATH = "/tmp/logs/module3"
%reload_ext tensorboard
%tensorboard --logdir  {TB_PATH}

CACHEPATH = os.path.expanduser('~/.local/data')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# pour mac
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# torch.set_default_dtype(torch.float32) # pour GPU mac
print(device)

cpu


# Prise en main des outils textes pour le deep-learning

L'utilisation la plus courante de l'apprentissage de représentation est de fournir une représentation utile pour la classification. Dans cette partie, nous utiliserons le corpus de critiques IMDB pour l'analyse de sentiment, un corpus qui contient des critiques de film et dont l'objectif est de prédire si une critique est  positive ou négative. Nous utiliserons pour la représentation d'une phrase le modèle moyen introduit ci-dessus : la moyenne des vecteurs de représentation des tokens qui la constitue. 

Explication du fonctionnement de la tokenization + padding sur un exemple jouet

=> On fera ensuite la même chose sur les données IMBD

In [13]:
tokenizer = get_tokenizer("basic_english") # recuperation d'un tokenizer (= séparateur de mots)

# test 1: tokenization d'une phrase:
phrs = ["the cat sat on the mat", "the dog eat", "the cat and the dog"]
phrs_tok = [tokenizer(t) for t in phrs]
print(phrs_tok)

# test 2: indexation des mots (ajout de mots speciaux)
tok_ind = torchtext.vocab.build_vocab_from_iterator(phrs_tok, specials=['<pad>','<oov>'], min_freq=0)
print("Dictionnaire : ", tok_ind.get_itos())

print("Taille du dictionnaire : ", len(tok_ind))

# test 3: indexation de la phrase 
phr_ind = [torch.tensor([tok_ind[t] for t in phr]) for phr in phrs_tok]
print("Phrase index : ", phr_ind)

# transformation inverse : phrase from index
print("reconstruction de la phrase : ", [tok_ind.get_itos()[i] for i in phr_ind[0]])

# test 4: padding: toutes les phrases doivent faire la même longueur
phr_ind_pad = pad_sequence(phr_ind, padding_value=tok_ind['<pad>'])
print("Phrase index pad: ", phr_ind_pad.T)

print("reconstruction de la phrase (pad) : ", [tok_ind.get_itos()[i] for i in phr_ind_pad.T[1]])


[['the', 'cat', 'sat', 'on', 'the', 'mat'], ['the', 'dog', 'eat'], ['the', 'cat', 'and', 'the', 'dog']]
Dictionnaire :  ['<pad>', '<oov>', 'the', 'cat', 'dog', 'and', 'eat', 'mat', 'on', 'sat']
Taille du dictionnaire :  10
Phrase index :  [tensor([2, 3, 9, 8, 2, 7]), tensor([2, 4, 6]), tensor([2, 3, 5, 2, 4])]
reconstruction de la phrase :  ['the', 'cat', 'sat', 'on', 'the', 'mat']
Phrase index pad:  tensor([[2, 3, 9, 8, 2, 7],
        [2, 4, 6, 0, 0, 0],
        [2, 3, 5, 2, 4, 0]])
reconstruction de la phrase (pad) :  ['the', 'dog', 'eat', '<pad>', '<pad>', '<pad>']


# DataLoader et Padding sur IMDB

Dans un corpus de texte, les phrases n'ont généralement pas toutes la même longueur. Cela pose problème pour la constitution des mini-batchs sous forme de tenseurs, et en général pour tout opération tensorielle dans le réseau. Une solution simple consiste à faire du *padding*, c'est-à-dire rajouter à la fin des phrases un caractère spécial  *<pad>* pour les compléter  jusqu'à la longueur voulue. Ainsi les mini-batchs sont constitués de phrases de même longueur mais qui contiennent le caractère *<pad>*. On utilise généralement un vecteur nul pour sa représentation (et l'index 0). Comme ce caractère est à la fin du texte, il ne gène pas en général pas pour la phase forward d'inférence. Il suffit alors de "masquer" lors du calcul du coût - c'est-à-dire ne pas prendre en compte - les erreurs dûes à ce caractère. Un argument de **nn.CrossEntropy** permet justement d'ignorer les coûts dues à un index de classe spécifique.
    
Il faut donc faire **DataLoader** spécifique pour un tel corpus en charge de faire du padding pour constituer des batchs de même taille. L'argument ```collate_fn(batch)``` d'un **DataLoader** permet de spécifier une fonction en charge d'aggréger la liste ```batch``` des exemples individuels samplés dans le **Dataset** et de rendre le minibatch associé. La fonction ```collate_batch_imdb(batch)```  permet :
* de convertir le texte de chaque élément du batch en liste d'indexes associés à chaque token du texte 
* de padder l'ensemble 
* de retourner le couple *(tenseur, étiquette)*. Vous pouvez utiliser la fonction ```torch.nn.utils.rnn.pad_sequence(l,batch_first,padding_value)``` qui permet de padder une liste de tenseurs (mettez ```batch_first=True``` pour que la fonction renvoie un tenseur dont la première dimension est le batch, la deuxième la longueur (et la troisième de dimension 1). 

In [14]:
# Chargement des données
imdb_train_iter, imdb_test_iter = IMDB(root=CACHEPATH,split=('train','test'))
imdb_train_list = list(imdb_train_iter)
imdb_test_list = list(imdb_test_iter)

In [15]:
# Construction du vocabulaire
tokenizer = get_tokenizer("basic_english") # recuperation d'un tokenizer (= séparateur de mots)
def yield_imdb():
    for _,l in imdb_train_list:
        yield tokenizer(l) # yield = generation à la volée != stockage dans une liste

# ajout de mots speciaux dans le vocabulaire => ils vont servir plus loin
imdb_vocab = torchtext.vocab.build_vocab_from_iterator(yield_imdb(),specials=['<pad>','<oov>'],min_freq=20)

In [16]:
# sur le vocabulaire IMDB
print("Taille du vocabulaire ", len(imdb_vocab))
print(imdb_vocab.get_itos()[:20]) # recup vocab (20 premiers mots)
print(imdb_vocab['the'])    # recup index from vocab

Taille du vocabulaire  13352
['<pad>', '<oov>', 'the', '.', ',', 'and', 'a', 'of', 'to', "'", 'is', 'it', 'in', 'i', 'this', 'that', 's', 'was', 'as', 'for']
2


In [17]:
## Mise en forme des textes, doit retourner le tenseur des indexes et le label
def collate_batch_imdb(batch):
    labels, texts = [],[]
    for (label,text) in batch:
        labels.append(1 if label==1 else 0)
        texts.append(torch.tensor([imdb_vocab[t]   for t in tokenizer(text) if t in imdb_vocab]))
    return pad_sequence(texts,batch_first=True, padding_value=imdb_vocab['<pad>']),torch.LongTensor(labels) 
    # attention à bien utiliser un mot spécial pour le padding


In [18]:
imdb_train_dataloader = DataLoader(imdb_train_list,batch_size=32,shuffle=True,collate_fn=collate_batch_imdb)
imdb_test_dataloader = DataLoader(imdb_test_list,batch_size=32,shuffle=True,collate_fn=collate_batch_imdb)

In [19]:
# vérification du résultat
next(iter(imdb_train_dataloader))

(tensor([[2157,   12,   64,  ...,    0,    0,    0],
         [  51,   26,  148,  ...,    0,    0,    0],
         [1293,  674,    8,  ...,    0,    0,    0],
         ...,
         [  14, 1306, 1698,  ...,    0,    0,    0],
         [  65,  359,    8,  ...,    0,    0,    0],
         [  51,   11,   45,  ...,    0,    0,    0]]),
 tensor([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
         0, 0, 0, 1, 1, 0, 1, 1]))

## <span class="alert-success">  Exercice : Apprentissage End-to-End </span>

Le premier réseau que nous allons étudier un réseau avec une couche d'embeddings pour représenter les tokens et de plusieurs couches de linéaires avec des ReLU comme fonction d'activation.
Il sera appris End-to-End dans un premier temps. 


Codez le réseau **ModeleMoyen** :  n'oubliez pas de prendre en compte le caractère **\<pad\>** dans la construction de la représentation de la phrase (en vrai cela n'a pas beaucoup d'importance dans ce cas précis, voyez-vous pourquoi ?).
Vous pouvez utiliser 100 dimensions pour la représentation, et deux couches de 50 et de 20 neurones suivies de ReLU.

<img src="./media/modaverage.png">

Entraînez le réseau, visualisez les embeddings obtenus.

In [20]:
EPOCHS = 10

def accuracy(yhat,y):
    assert len(y.shape)==1 or y.size(1)==1
    return (torch.argmax(yhat,1).view(y.size(0),-1)== y.view(-1,1)).float().mean()


class ModeleMoyen(nn.Module):
    # bien comprendre les arguments
    def __init__(self,vocab_size,input_dim,output_dim,layers,padding_idx,pre_trained=None):
        """
        vocab_size : taille vocabulaire
        input_dim : taille des embeddings
        output_dim : nb classes (ATTENTION, même dans le cas binaire, mettre 2 si loss = crossentropy)
        layers : liste des dimensions des couches cachées [h_layer1, h_layer2, ...]
        padding_idx : indice du mots de padding (à ne pas apprendre => emb_pad = [0,0,...0])
        """
        super().__init__()
        # 0) save useful params
        self.padding_idx = padding_idx # reference du mot special pad
        # 1) il faut construire la table des représentation dans le réseau
        if pre_trained is None:
            self.embedding = nn.Embedding(vocab_size,input_dim,padding_idx=padding_idx) # pad n'est pas appris => 0
        else:
            self.embedding = nn.Embedding.from_pretrained(pre_trained, freeze=True, padding_idx=padding_idx)
        # 2) construire les nlayers couches Linear + RELU
        # 3) les mettre dans un Sequential
        #       note: si listecouches = [couche1, couche2, ...] => self.fc = nn.Sequential(*listecouches)
        ##  TODO 


    def forward(self,x):
        # 1) recupération des embeddings des x
        # 2) moyenne = somme / nombre de vrais mots (pas les pads)
        # 3) forward du résultat
        ##  TODO 

# tjs le même code (ou presque)
def train(model,epochs,train_loader,test_loader):
    writer = SummaryWriter(f"{TB_PATH}/{model.name}")
    optim = torch.optim.Adam(model.parameters(),lr=1e-3)
    model = model.to(device)
    print(f"running {model.name}")
    loss = nn.CrossEntropyLoss() # ATTENTION: nout = 2 pour un problème binaire si crossentropie
    for epoch in tqdm(range(epochs)):
        cumloss, cumacc, count = 0, 0, 0
        model.train()
        for x,y in train_loader:
            optim.zero_grad()
            x,y = x.to(device), y.to(device)
            yhat = model(x)
            l = loss(yhat,y)
            l.backward()
            optim.step()
            cumloss += l*len(x)
            cumacc += accuracy(yhat,y)*len(x) # calcul de la moyenne (avec des batchs inegaux)
            count += len(x)
        writer.add_scalar('loss/train',cumloss/count,epoch)
        writer.add_scalar('accuracy/train',cumacc/count,epoch)
        if epoch % 1 == 0: # on peut changer le pas pour gagner un peu de temps de calcul
            model.eval()
            with torch.no_grad():
                cumloss, cumacc, count = 0, 0, 0
                for x,y in test_loader:
                    x,y = x.to(device), y.to(device)
                    yhat = model(x)
                    cumloss += loss(yhat,y)*len(x)
                    cumacc += accuracy(yhat,y)*len(x)
                    count += len(x)
                writer.add_scalar(f'loss/test',cumloss/count,epoch)
                writer.add_scalar('accuracy/test',cumacc/count,epoch)


In [21]:
## Création et entraînement du modèle moyen, visualisation des embeddings
# 1) trouver les bons arguments pour construire votre réseau
# 2) nommer le réseau (comme d'habitude)
# 3) lancer train

##  TODO 



running modeleMoyen_Mon Jan  9 13:35:17 2023


100%|██████████| 10/10 [03:47<00:00, 22.70s/it]


### Evaluation & Visualisation des embeddings de mots appris

- les courbes d'apprentissage (et validation) doivent être très bonnes
- les représentations de mots ont l'air d'avoir un peu de sens... Mais ce n'est pas très impressionnant

In [ ]:
# test d'un exemple custom
phr = "this movie is awful" 
# phr = "the movie is great"
input = torch.tensor([[imdb_vocab[t]   for t in tokenizer(phr) if t in imdb_vocab]])
print(input)
print(modeleMoyen(input))

In [ ]:
# recherche de la première erreur

for lab,txt in imdb_train_list:
    input = torch.tensor([[imdb_vocab[t]   for t in tokenizer(txt) if t in imdb_vocab]])
    pred = modeleMoyen(input).argmax().item()
    # print(lab,pred)
    if lab != pred:
        print(lab,pred,txt)
        break

In [ ]:
# visualisation des embeddings appris (50 premiers mots)
writer = SummaryWriter(f"{TB_PATH}/{modeleMoyen.name}")
writer.add_embedding(modeleMoyen.embedding.weight[:50],metadata=imdb_vocab.get_itos()[:50],global_step=3)


## Représentations pré-apprises et fine-tuning 

Une option de **nn.Embeddings** permet de charger des représentations pré-entraînées. Dans ce cas, par défaut, la couche de représentation est gelée : elle n'est pas mise à jour durant l'apprentissage. On peut *fine-tuner* les représentations apprises, c'est-à-dire les adapter à notre tâche, en remettant à vrai le flag ```requires_grad``` du tenseur de représentation.

Implémentez et testez les deux réseaux en utilisant ```fasttext.simple.300d```. 

In [24]:
## Création du réseau modèle moyen à partir des représentation fasttext.simple.300d
## Entraînement avec et sans fine-tuning.

fasttext = torchtext.vocab.FastText('simple')
fast_imdb_vectors = torch.stack([fasttext[m] for m in imdb_vocab.get_itos()])

In [ ]:
modeleMoyenPretrained = ModeleMoyen(len(imdb_vocab),300,2,[50,20],imdb_vocab["<pad>"],fast_imdb_vectors)
modeleMoyenPretrained.name = "pretrained_m"+time.asctime()
train(modeleMoyenPretrained,EPOCHS,imdb_train_dataloader,imdb_test_dataloader)

# bien vérifier les performances dans tensorboard
# Eventuellement comparer les embeddings dans tensorboard
# reflechir sur le pourcentage des paramètres qui ont été mis à jour pendant l'apprentissage


In [ ]:
modeleMoyenPretrained_fine = ModeleMoyen(len(imdb_vocab),300,2,[50,20],imdb_vocab['<pad>'],fast_imdb_vectors)
## Permet de réactiver l'apprentissage des représentations
modeleMoyenPretrained_fine.embedding.weight.requires_grad = True # fine-tuning
modeleMoyenPretrained_fine.name="pretrained_fine_m"+time.asctime()
train(modeleMoyenPretrained_fine,EPOCHS,imdb_train_dataloader,imdb_test_dataloader)


# Convolution et texte

Le modèle précédent est relativement pauvre : le texte est représenté uniquement par le barycentre des représentations des tokens, ce qui élimine du coup toute notion de sequentialité. Une première manière de traiter la séquentialité est d'utiliser des couches convolutionnelles  et de pooling comme en image. Cette fois cependant, la convolution sera en une dimension et non pas deux.

L'interaction entre l'opérateur de convolution et de max-pooling peut être vue de la manière suivante : 

    1. La convolution permet de détecter un ensemble de motifs (par exemple "un bon film", "très bon film", "un film déplorable") sur des séquences courtes qui dépend de la taille du noyau (dans cet exemple, 3 mots).
    2. Le max-pooling permet de résumer l'information capturée par les filtres sur une sous-séquence de taille plus grande. 
    
Ces opérations sont répétées comme en image plusieurs fois afin de détecter des motifs de taille et de complexité croissante. Par exemple, on peut détecter "j'ai beaucoup"/"j'ai énormément"/"ai pas beaucoup" et "ce film"/"cet acteur" (4 filtres) sur la première couche de convolution. La seconde couche de convolution va combiner les informations pour détecter un jugement sur un film ou un acteur.

La dernière couche est généralement un maximum global (sur chaque filtre de sortie), suivie d'un classifieur linéaire.

Codez une architecture convolutive pour les données IMDB. 

In [25]:
class Scope:
    """Permet de savoir quelle est la portée d'une couche"""
    def __init__(self):
        self.width = 1
        self.stride = 1

    def __call__(self, width, stride=1):
        self.width = (width - 1) * self.stride + self.width
        self.stride = self.stride * stride
        return self

    def __repr__(self):
        return "(w=%d,s=%d)" % (self.width, self.stride)

class ModelCNN(torch.nn.Module):
    def __init__(self, vocab_size, input_dim, output_dim, nfeat,padding_idx,pre_trained=None):
        super().__init__()
        self.nfeat = nfeat

        if pre_trained is None:
            self.embedding = nn.Embedding(vocab_size,input_dim,padding_idx=padding_idx)
        else:
            self.embedding = nn.Embedding.from_pretrained(pre_trained, freeze=True, padding_idx=padding_idx)

        self.convolutions = nn.Sequential(
            nn.Conv1d(input_dim, 100, 3), # in, out, kernelsize
            nn.MaxPool1d(3,1), # kernelsize, stride (=agregation 3 par 3) => longueur = longueur -2, nfiltres = cst
            nn.ReLU(),
            nn.Conv1d(100, 100, 4),
            nn.MaxPool1d(4,1),
            nn.ReLU(),
            nn.Conv1d(100, self.nfeat, 6),
            nn.MaxPool1d(4,1)
        )

        self.scope = Scope()
        for m in self.convolutions:
            if isinstance(m, torch.nn.modules.conv.Conv1d):
                self.scope = self.scope(m.kernel_size[0], m.stride[0])
            elif isinstance(m, torch.nn.modules.pooling.MaxPool1d):
                self.scope = self.scope(m.kernel_size, m.stride)
        print("Scope %s", self.scope)

        self.fc = nn.Sequential(
            nn.Linear(self.nfeat, output_dim),
        )

    def convolution_fn(self, x: torch.Tensor):
        x = self.embedding(x).transpose(1,2)
        y = self.convolutions(x)
        return y

    def forward(self, x: torch.Tensor):
        y = self.convolution_fn(x)
        z = self.fc(y.max(2)[0])
        return z


In [26]:
modelCNN = ModelCNN(len(imdb_vocab),300,2,20,imdb_vocab["<pad>"],fast_imdb_vectors)
modelCNN.name = "CNN"+time.asctime()

Scope %s (w=19,s=1)


In [28]:
train(modelCNN,10,imdb_train_dataloader,imdb_test_dataloader)

running CNNMon Jan  9 13:47:26 2023


100%|██████████| 10/10 [2:53:09<00:00, 1038.93s/it] 


In [ ]:
modelCNN_fine = ModelCNN(len(imdb_vocab),300,2,20,imdb_vocab["<pad>"],fast_imdb_vectors)
modelCNN_fine.embedding.freeze = False
modelCNN_fine.name = "CNN-fine"+time.asctime()
train(modelCNN_fine,10,imdb_train_dataloader,imdb_test_dataloader)

In [ ]:
import os

def save_model(model,fichier): # pas de sauvegarde de l'optimiseur ici
      """ sauvegarde du modèle dans fichier """
      state = {'model_state': model.state_dict()}
      torch.save(state,fichier) # pas besoin de passer par pickle
 
def load_model(fichier,model):
      """ Si le fichier existe, on charge le modèle  """
      if os.path.isfile(fichier):
          state = torch.load(fichier)
          model.load_state_dict(state['model_state'])

In [ ]:
# sauvegarde du réseau (économie de 30 minutes :)

path = "/Users/vguigue/Documents/Cours/Agro-IODAA/deep/notebooks/"

fichier = path+"model/modelCNN"
save_model(modelCNN,fichier)

fichier = path+"model/modelCNN_fine"
save_model(modelCNN_fine,fichier)

# vous pouvez utiliser les formules symmétriques pour le chargement

# modelCNN = ModelCNN(len(imdb_vocab),300,2,20,imdb_vocab["<pad>"],fast_imdb_vectors).to(device)
# modelCNN.name ="Conv-trained"
# modelCNN_fine = ModelCNN(len(imdb_vocab),300,2,20,imdb_vocab["<pad>"],fast_imdb_vectors).to(device)
# modelCNN_fine.name ="Conv_fine-trained"
# load_model("model/modelCNN", modelCNN)
# load_model("model/modelCNN_fine", modelCNN_fine)

#### Afin d’étudier ce que fait le CNN, nous allons nous intéresser à la dernière couche avant le maximum global ; plus particulièrement, nous allons chercher les sous-séquences (dans le jeu de train) qui activent le plus chaque filtre de sortie.

Pour cela, il faut tout d’abord déterminer à quelle position (dans le texte) correspond chaque sortie. Par exemple, si on considère une convolution avec une taille de noyau 3 et un stride de 1, alors la 1ème sortie correspondra au texte entre les positions 1 et 3, la 2ème à celui entre les positions 2 et 4, etc. Si on ajoute un max-pooling (noyau de taille 2, stride 2), alors la 1ère sortie correspond au texte entre les positions 1 et 4, la 2ème aux positions 3 et 6, etc.

Il faut maintenant généraliser. Pour cela, nous allons considér que la ième opération (convolution/pooling) est définie seulement par la taille du noyau $w_i$ (*kernel width*) et le déplacement $s_i$ (*stride*). Nous nous intéressons aux deux valeurs qui caractérisent l’ensemble des transformations jusqu’a l’opération $i−1$ : 
    
* La longueur des entrées correspondant à une sortie $W_{i−1}$.
* Le déplacement $S_{i−1}$ dans les entrées correspondant à un déplacement unitaire dans les sorties.

Donner la formule de récurrence qui permet, étant donné $W_i$ , $S_i$ de déterminer $W_{i+1}$
et $S_{i+1}$ sachant $w_{i+1}$ et $s_{i+1}$.

Soit $(y_1,\ldots, y_L )$ la sortie du CNN. Une fois ce calcul fait, donnez la formule qui, étant
donné la position j de la sortie $y_j$ , permet de déterminer les indices correspondant
dans la séquence d’entrée.

Finalement, parcourez les données du jeu de train, et trouvez les sous-séquences qui
activent le plus chaque caractéristique de sortie.

In [ ]:
import heapq
# Un individu (texte + valeur)
class Sample:
    def __init__(self, value, text):
        self.value = value
        self.text = text

    def __repr__(self):
        return "Sample(%g)" % self.value

    def __lt__(self, other):
        return self.value < other.value

K = 20
topk = [[] for i in range(modelCNN_fine.nfeat)]

with torch.no_grad():
        sampler = imdb_train_dataloader
        for batch in sampler:
            if batch[0].shape[1] < modelCNN_fine.scope.width:
                continue
            
            # y a une taille batch x nfeat x lmax
            y = modelCNN_fine.convolution_fn(batch[0].to(device))
            bs, _nfeat, lmax = y.shape
            assert _nfeat == modelCNN_fine.nfeat

            # On cherche l'argmax sur toutes les activations
            # On transpose d'abord (nfeat x batch x lmax)
            yt = y.transpose(0,1).contiguous().view(modelCNN_fine.nfeat,-1)
            _, indices = yt.topk(K, 1)
            indices = indices.to('cpu')
            
            for i in range(modelCNN_fine.nfeat):
                for k in range(indices.shape[1]):
                    # On retrouve le batch et la position
                    batch_ix = int(indices[i, k] // lmax)
                    pos = int(indices[i, k] % lmax)

                    # On retrouve le texte correspondant
                    text = batch[0][batch_ix, pos * modelCNN_fine.scope.stride: pos*modelCNN_fine.scope.stride + modelCNN_fine.scope.width]
                    sample = Sample(yt[i, indices[i, k]], text)

                    if len(topk[i]) < K:
                        heapq.heappush(topk[i], sample)
                    elif topk[i][0].value < sample.value:
                        heapq.heapreplace(topk[i], sample)

for i in range(modelCNN_fine.nfeat):
    print("==== Feat. %d ==== " % i)
    for sample in sorted(topk[i], reverse=True):
        print("%.2f: %s" % (float(sample.value), " ".join(imdb_vocab.get_itos()[int(tid)] for tid in sample.text)))



# Construction du sujet à partir de la correction

In [ ]:
###  TODO )"," TODO ",\
    txt, flags=re.DOTALL))
f2.close()

### </CORRECTION> ###